# EN einhliðamálheild
Í þessu reikniriti eru enskar einhliðamálheildir lesnar og staðlaðar svo ein setning sé í hverri línu. Málheildirnar eru eftirfrandi. Sameinum í eitt skjal.

| Nafn | #línur| stærð | ath |
| --- | --- | --- | --- |
| newscrawl 2018 en | 18,1M | 2 GB | - |
| UN corpus EN | 27,4M | 3,4GB | - |
| Europarl v9 | 2,3M | 321 MB | - |
| Librispeech 27 | 48,2M | 4,3GB | - | - |
| Wikipedia Google en | 25,8M | 3,0GB | - |
| --- | --- | --- | --- |
| Heild | 121,9M | 13,1GB | - |

In [3]:
# The location of en-mono to read
source_dir = '/work/haukurpj/data/raw/en-monolingual'

# The location of where to write the results
target_dir = '/work/haukurpj/data/formatted/en-monolingual'

from glob import glob
from pprint import pprint

In [4]:
import pathlib

source_dir = pathlib.Path(source_dir)
target_dir = pathlib.Path(target_dir)
assert source_dir.exists()
if not target_dir.exists():
    target_dir.mkdir()

In [17]:
def read_file(path):
    corpus = []
    with path.open() as f_in:
        return [line for line in f_in]

## Europarl EN
Þessi gögn líta vel út og ekkert þarf að gera.

In [5]:
euro_parl = source_dir.joinpath('europarl-v9.en')
!wc -l {euro_parl}
!head {euro_parl}

2295044 /work/haukurpj/data/raw/en-monolingual/europarl-v9.en
Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday, 15 December 2000.
Statements by the President
Ladies and gentlemen, on Saturday, as you know, an earthquake struck Central America once again, with tragic consequences.
This is an area which has already been seriously affected on a number of occasions since the beginning of the twentieth century.
The latest, provisional, figures for victims in El Salvador are already very high.
There are 350 people dead, 1 200 people missing, the area is completely devastated and thousands of homes have been destroyed throughout the country.
The European Union has already shown its solidarity by sending a rescue team to the area, whilst financial assistance from the Union and Member States has been, or is in the process of being, released and I am able to inform you that some groups in the European Parliament have requested that this issue

In [18]:
euro_parl = read_file(source_dir.joinpath('europarl-v9.en'))

## News crawl 2018
Lítur vel út - ekkert sem þarf að gera.

In [7]:
news = source_dir.joinpath('news.2018.en.shuffled.deduped')
!wc -l {news}
!head {news}

18113311 /work/haukurpj/data/raw/en-monolingual/news.2018.en.shuffled.deduped
He owns the Orphaned Wildlife Center with his wife, Susan.
Qatar also made a loose commitment not to launch flights to the United States from Europe or other non-Qatari cities that would create yet more competition for U.S. carriers.
The fact there's a banker pig as well is kind of amusing.
Berlinger was the "gold standard" and "a huge hole that we have to fill."
"From tonight on, I'm sorry for them, but they are going to be different," coach Didier Deschamps said.
:: The top 10 beaches in the UK according to TripAdvisor are:
Morgan, who has a lengthy criminal history including drugs offences, had not been investigated for suspected terrorism before.
But despite the major disruption caused and resulting financial bill, Judge Philip Statman said Khonde had already been sufficiently punished by being on remand since his arrest and serving the equivalent of 14 months' custody.
Just type Bono and hypocrite into T

In [19]:
news = read_file(source_dir.joinpath('news.2018.en.shuffled.deduped'))

## Wikipedia
Þessi gögn eru í `.csv`, við þurfum að laga það.

In [10]:
import pandas as pd
wiki = source_dir.joinpath('documents_utf8_filtered_20pageviews.csv')
wiki_df = pd.read_csv(str(wiki), sep=",", header=None)
wiki_df.head()

,0,1
0,wikipedia-23885690,Research Design and Standards Organization T...
1,wikipedia-23885928,The Death of Bunny Munro The Death of Bunny ...
2,wikipedia-23886057,Management of prostate cancer Treatment for ...
3,wikipedia-23886425,Cheetah reintroduction in India Reintroducti...
4,wikipedia-23886491,Langtang National Park The Langtang National...


In [14]:
wiki = wiki_df[1].values.tolist()
wiki[:2]

[' Research Design and Standards Organization  The Research Design and Standards Organisation (RDSO) is an ISO 9001 research and development organisation under the Ministry of Railways of India, which functions as a technical adviser and consultant to the Railway Board, the Zonal Railways, the Railway Production Units, RITES and IRCON International in respect of design and standardisation of railway equipment and problems related to railway construction, operation and maintenance. History. To enforce standardisation and co-ordination between various railway systems in British India, the Indian Railway Conference Association (IRCA) was set up in 1903. It was followed by the establishment of the Central Standards Office (CSO) in 1930, for preparation of designs, standards and specifications. However, till independence in 1947, most of the designs and manufacture of railway equipments was entrusted to foreign consultants. After independence, a new organisation called Railway Testing and R

Núna er skjalið með eina grein í hverri línu.

In [15]:
import nltk
import re

wiki_sent = []

for line in wiki:
    line = line.strip()
    for sent in nltk.sent_tokenize(line):
        wiki_sent.append(sent)

In [16]:
wiki_sent[:5]

['Research Design and Standards Organization  The Research Design and Standards Organisation (RDSO) is an ISO 9001 research and development organisation under the Ministry of Railways of India, which functions as a technical adviser and consultant to the Railway Board, the Zonal Railways, the Railway Production Units, RITES and IRCON International in respect of design and standardisation of railway equipment and problems related to railway construction, operation and maintenance.',
 'History.',
 'To enforce standardisation and co-ordination between various railway systems in British India, the Indian Railway Conference Association (IRCA) was set up in 1903.',
 'It was followed by the establishment of the Central Standards Office (CSO) in 1930, for preparation of designs, standards and specifications.',
 'However, till independence in 1947, most of the designs and manufacture of railway equipments was entrusted to foreign consultants.']

## UN Corpus en 
Þessi málheild byggist á mörgum .tei skjölum. Við þurfum að lesa öll skjölin og sameina í eitt stórt skjal. Þar sem flest skjöl byrja á fundarlýsingu og öðrum setningum sem eru ekki sérlega góðar setningar. Við byrjum því að lesa setningar eftir setningu númer 40.

In [21]:
un_dir = source_dir.joinpath('UNv1.0-TEI')
xml_files = glob(f'{un_dir}/**/*.xml', recursive=True)
xml_files = [pathlib.Path(xml_file) for xml_file in xml_files]
len(xml_files)

159328

In [22]:
!tail -n 60 {xml_files[0]}

      <p id="46">
        <s id="46:1" lang="en">Ms. Dicko (Mali) 11</s>
      </p>
      <p id="47">
        <s id="47:1" lang="en">16. Having obtained the required majority and the largest number of votes, Mr. Brillantes (Philippines), Mr. Ibarra Gonzalez (Guatemala), Mr. Kariyawasam (Sri Lanka) and Mr. Tall (Senegal) were elected members of the Committee for a term beginning 1 January 2010.</s>
      </p>
      <p id="48">
        <s id="48:1" lang="en">17. The Chairperson said that, since only four candidates had obtained the required majority, a second ballot, restricted to the two candidates who had obtained the next largest number of votes, would be held to fill the remaining vacancy, in accordance with rule 15 of the rules of procedure.</s>
      </p>
      <p id="49">
        <s id="49:1" lang="en">18. A vote was taken by secret ballot.</s>
      </p>
      <p id="50">
        <s id="50:1" lang="en">Number of valid ballot papers: 41</s>
      </p>
      <p id="51">
        <s 

In [24]:
from xml.etree import ElementTree as ET

def read_un_tei_file(path: pathlib.Path, min_p_id=40):
    sentences = []
    try: 
        root = ET.parse(str(path)).getroot()
        # We gather all the paragraphs from the body, avoiding the divs
        for paragraph_node in root.findall('.//body//p'):
            # we only take sentences which have an "id" high enough.
            if int(paragraph_node.attrib['id']) < min_p_id:
                continue
            for sentence_node in paragraph_node.findall('.//s'):
                sentence = sentence_node.text
                sentences.append(sentence)
    except:
        # We just skip the file
        pass
    return sentences

In [25]:
print(read_un_tei_file(xml_files[0]))

['Mr. Kariyawasam (Sri Lanka) 28', 'Mr. Tall (Senegal) 22', 'Mr. Sevim (Turkey) 18', 'Mr. Carrión Mena (Ecuador) 18', 'Ms. Miller-Stennett (Jamaica) 17', 'Mr. Taghizade (Azerbaijan) 14', 'Ms. Dicko (Mali) 11', '16. Having obtained the required majority and the largest number of votes, Mr. Brillantes (Philippines), Mr. Ibarra Gonzalez (Guatemala), Mr. Kariyawasam (Sri Lanka) and Mr. Tall (Senegal) were elected members of the Committee for a term beginning 1 January 2010.', '17. The Chairperson said that, since only four candidates had obtained the required majority, a second ballot, restricted to the two candidates who had obtained the next largest number of votes, would be held to fill the remaining vacancy, in accordance with rule 15 of the rules of procedure.', '18. A vote was taken by secret ballot.', 'Number of valid ballot papers: 41', 'Number of representatives voting: 41', 'Mr. Sevim (Turkey) 24', 'Mr. Carrión Mena (Ecuador) 17', '19. Having obtained the required majority, Mr. S

In [27]:
un = []
for xml_file in xml_files:
    sentences = read_un_tei_file(xml_file, min_p_id=50)
    for sentence in sentences:
        if sentence:
            un.append(sentence + '\n')

## Libri Speech 27

Þessi málheild er samansafn af bókum og inniheldur mikið af tómum línum. Einnig er hver bók í sér skrá.

In [29]:
libri_dir = source_dir.joinpath('librispeech').joinpath('corpus')
txt_files = glob(f'{libri_dir}/**/*.txt', recursive=True)
txt_files = [pathlib.Path(txt_file) for txt_file in txt_files]
len(txt_files)

14476

In [30]:
!tail -n 10 {txt_files[0]}




Transcriber's Notes

Passages in italics are indicated by _underscores_.






In [31]:
!head -n 50 {txt_files[0]}

book was produced from scanned images of public domain
material from the Google Print project.)








The Outlook:

Uncle Sam's Place and
Prospects in International
Politics

[Illustration]

Newton Macmillan.




THE OUTLOOK:

UNCLE SAM'S PLACE AND PROSPECTS IN
INTERNATIONAL POLITICS.

A Paper

READ BEFORE THE FORTNIGHTLY CLUB, OSWEGO, N. Y.,
MAY 2, 1899,

BY

NEWTON MACMILLAN.


ORDERED PUBLISHED BY THE FORTNIGHTLY CLUB.

1899.




PRESS OF R. J. OLIPHANT,
OSWEGO, N. Y.





In [32]:
import nltk
import re

def read_libri_file(p_in):
    sentences = []
    buffer = []
    with p_in.open() as f_in:
        for line in f_in:
            line = line.strip()
            if not line:
                if buffer:
                    for sent in nltk.sent_tokenize(" ".join(buffer)):
                        sentences.append(sent)
                    buffer = []
            else:
                buffer.append(line)
    if buffer:
        for sent in nltk.sent_tokenize(" ".join(buffer)):
            sentences.append(sent)
        buffer = []
    return sentences

In [33]:
read_libri_file(txt_files[0])[:16]

['book was produced from scanned images of public domain material from the Google Print project.)',
 'The Outlook:',
 "Uncle Sam's Place and Prospects in International Politics",
 '[Illustration]',
 'Newton Macmillan.',
 'THE OUTLOOK:',
 "UNCLE SAM'S PLACE AND PROSPECTS IN INTERNATIONAL POLITICS.",
 'A Paper',
 'READ BEFORE THE FORTNIGHTLY CLUB, OSWEGO, N. Y., MAY 2, 1899,',
 'BY',
 'NEWTON MACMILLAN.',
 'ORDERED PUBLISHED BY THE FORTNIGHTLY CLUB.',
 '1899.',
 'PRESS OF R. J. OLIPHANT, OSWEGO, N. Y.',
 'THE OUTLOOK:',
 "UNCLE SAM'S PLACE AND PROSPECTS IN INTERNATIONAL POLITICS."]

Við náum ekki alveg textanum réttum með NLTK. En við köllum það gott.

In [34]:
libri = []
for txt_file in txt_files:
    for sentence in read_libri_file(txt_file):
        libri.append(sentence + '\n')

## Sameinum í eitt skjal

In [35]:
corpora = [
    euro_parl,
    news,
    wiki_sent,
    un,
    libri
]

with target_dir.joinpath('mono.en').open('w+') as f_out:
    for corpus in corpora:
        for line in corpus:
            f_out.write(line)
    !wc -l {target_dir.joinpath('mono.en')}

94541108 /work/haukurpj/data/formatted/en-monolingual/mono.en
